In [ ]:
%pylab inline

In [ ]:
import cPickle as pickle
import gzip

In [ ]:
#data = load('PMPulse_1_6_Vmon-1.pkl')
#pickle.dump(data,gzip.open('PMPulse_1_6_Vmon-1.pklz','wb'))

In [ ]:
# Load Data
data = pickle.load(gzip.open('PMPulse_1_6_Vmon-1.pklz','rb'))
dt = 1/20000. #20 kHz sampling rate
T = 8. #sec
time = linspace(0,T,T/dt)
stim_currents = arange(250,601,50)
stim_start = int(1/dt)
stim_end = int(5/dt)

In [ ]:
print data.shape
#the first dimension are sweeps with different strengths of depolarization currents (50pA steps), second dimension is time
#stim_current = ???


# select the voltage trace of the last sweep
sweep_idx = -1
V = data[sweep_idx]

## Part I

__1)__ 
- How long is the complete voltage trace in seconds?
- How long was the stimulation?
- Plot the  complete voltage trace
- Plot only the period of the stimulation
- Set the range of the yaxis to (-75mV, 20mV).

__2)__
Using the voltage trace, determine the times at which the cell fired action potentials.
- How many action potentials did you detect?
- What is the average firing rate (i.e. action potentials per second) of this neuron?
- Determine the Inter-Spike-Interval distribution

In [ ]:
# we need the first element of threshold crossing for each spike
# convert the bool array to int (0 and 1), diff is the elementwise difference
threshold = 0.02 # V
V_above_threshold = data>threshold
V_above_th_diff = diff(V_above_threshold.astype(int), axis=-1) == 1
V_above_th_diff = c_[zeros(V_above_th_diff.shape[0]),V_above_th_diff]
# 1 for spike onset and -1 for spike offset; select only onsets
spike_bins = V_above_th_diff == 1 
N_spikes = spike_bins.sum(-1)

In [ ]:
print 'number of spikes:', N_spikes
print 'firing rate:', N_spikes/T, 'Hz'

In [ ]:
spike_times = [find(sb)*dt for sb in spike_bins]
ISIs =        [diff(st) for st in spike_times]
boxplot(ISIs, labels=stim_currents);

In [ ]:
plot(ISIs[sweep_idx], '.')

__3)__ Spike Triggered Average (STA)

Now determine the average waveform of an action potential. To this end you need to get  all AP onset times to obtain a cut-out of about +/- 10 ms around the spike time and average across all the cutouts.

- Add to the plot the average waveform ± 1 standard deviation across all APs.

In [ ]:
cutout = int(10e-3/dt) #10 ms

window = arange(-cutout,+cutout)
# create an array with the indices of all cut-outs around spikes onset
# first, repeat the cut-out window N_spike times
spike_select_idx = tile(window,(N_spikes[sweep_idx],1)).T
# then, add a spike index to every window
spike_select_idx = spike_select_idx + find(spike_bins[sweep_idx])
# finally in case the first or the last spike are to close to boundary,
# clip the indices to stay within limits
spike_select_idx = spike_select_idx.clip(0,len(V)-1)

print window.shape, N_spikes, spike_select_idx.shape

In [ ]:
# now we can use the index array to get the voltage for all the cut-outs
V_STA = V[spike_select_idx].mean(1)
plot(linspace(-10,10,cutout*2), V_STA, 'k', lw=2)
plot(linspace(-10,10,cutout*2), V[spike_select_idx], color='gray', zorder=-1);
xlabel('Time [ms]')
ylabel('V [V]')


__4)__

- Get the subthreshold membrane voltage. To do so exclude the AP cutouts (+/- 10 ms around the spike time) from to voltage trace. <BR>Hint: you might use a `masked array` to mask the spikes from the voltage trace<br>i.e. `V_subthres = np.ma.masked_array(V,mask=mask_spikes`).

- Plot 500ms of subthreshold voltage before stimulation and 500 ms of subthreshold voltage during stimulation

- What is the mean and variance of the subthreshold membrane potential distribution
    - before stimulation
    - during stimulation
    - after stmulation? (discard the first 250ms after stimulation offset)
   
- compare the subthreshold membrane potential distribution before/during and after stimulation.    

In [ ]:
# mask the membrane potential in a window around the spike times
wdw_size = int(round(0.01/dt))
mask_spikes = array([convolve(sb, ones(wdw_size), mode='same') for sb in spike_bins])

In [ ]:
V_subthres = ma.masked_array(data,mask=mask_spikes)

In [ ]:
plot(time, V_subthres[sweep_idx])
xlim(0.5,1.5)

In [ ]:
boxplot([v.compressed()*1e3 for v in V_subthres[:,:stim_start]], showmeans=True);

In [ ]:
boxplot([v.compressed()*1e3 for v in V_subthres[:,stim_start:stim_end]], showmeans=True);

In [ ]:
boxplot([v.compressed()*1e3 for v in V_subthres[:,stim_end+5000:]], showmeans=True);

__5)__

The standard integrate-and-fire (I&F) model of a neuron assumes a fixed voltage threshold for initiating a spike and a fixed reset potential. Can you determine the reset potential and a fixed voltage threshold that determines the onset of each spike?


- assume the reset potential is given by the minimum in the voltage after each spike. determine the mean and standart deviation of the reset potential. Does it change over time?


- Now look at the distribution of the derivative of the membrane voltage ($\frac{d}{dt}V_m$)
    * assume that the membrane potential crosses threshold when $\frac{d}{dt}V_m$ is above a critical value. What is a good choice for the critical value of $\frac{d}{dt}V_m$?
    * Plot the distribution of thresholds for individual spikes. Does the threshold change over time?
    
    * Now, instead of a critical value, assume that the threshold is given by the maxima of $\frac{d}{dt}V_m$ and repeat the analysis.    